# About
- Setup a simple RAG with the transcripts
- Test out some of the questions that I generated in notebook 3
- Explore whether I am happy with the answers or if the answers should be more detailed


# Imports and Settings

In [44]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv


from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS, Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub



from langchain_core.runnables import (
    ConfigurableField,
    RunnableBinding,
    RunnableLambda,
    RunnablePassthrough,
    RunnableParallel
)

pd.set_option('display.max_colwidth', None)  # Or use a large number if 'None' does not work in some environments
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all row

load_dotenv()
OPENAI_APIKEY = os.environ['OPENAI_APIKEY']

# Instantiate Models

In [35]:
embeddings_model = OpenAIEmbeddings(api_key=OPENAI_APIKEY, model='text-embedding-3-large', max_retries=100, chunk_size=16, show_progress_bar=False)
chat_model_3 = ChatOpenAI(api_key=OPENAI_APIKEY, temperature=0.5, model='gpt-3.5-turbo-0125')
chat_model_4 = ChatOpenAI(api_key=OPENAI_APIKEY, temperature=0.5, model='gpt-4-turbo-2024-04-09')

# Load Data

In [10]:
df_transcripts = pd.read_csv('../data/transcripts_all_2024-04-10_cleaned')
df_transcripts    
## I SHOULD REMOVE THE UNNAMED COLUMNS IN THIS FILE OTHERWISE THEY END UP AS METADATA##

,Unnamed: 0.1,Unnamed: 0,videoId,title,description,publishedAt,date,year,transcript
0,0,0,r3PuCQ8CxTc,Benefits and Side Effects of the Pneumonia Vac...,Randomized controlled trials have found that p...,2024-04-10T11:59:52Z,2024-04-10,2024,"""Benefits and Side Effects of the Pneumonia Va..."
1,2,2,oa6UtySJKFE,Benefits and Side Effects of the Flu Vaccine,Flu shots can prevent more than just the flu. ...,2024-04-08T11:59:55Z,2024-04-08,2024,"In this 3-video series, I show the science beh..."
2,3,3,wZLgy4dvM1Y,New Sponsorship: Big Broccoli,Help keep us ad- and sponsorship-free by makin...,2024-04-07T15:59:56Z,2024-04-07,2024,I’m Dr. Michael Greger and ever since I starte...
3,4,4,Et0lozAIbI8,Friday Favorites: Removing Warts with Duct Tape,Duct tape beat out cryotherapy (freezing) and ...,2024-04-05T11:59:52Z,2024-04-05,2024,You can find home remedies for all sorts of ai...
4,5,5,EoAaCdlK6CU,Podcast: Hot Flashes,Turns out that hot flashes are not inevitable....,2024-04-04T12:59:57Z,2024-04-04,2024,I hear from lots of people every day who are c...
...,...,...,...,...,...,...,...,...,...
2564,2592,2592,P_X3exQtuGA,The Healthiest Herbal Tea,New subscribers to our e-newsletter always rec...,2011-02-08T14:59:42Z,2011-02-08,2011,"""The Healthiest Herbal Tea"" Walking through th..."
2565,2593,2593,ce-pvksbiwM,Update on Yerba Maté,New subscribers to our e-newsletter always rec...,2011-02-08T14:57:18Z,2011-02-08,2011,"""Update on Yerba Maté"" And finally, what about..."
2566,2594,2594,1Yb5MjU38ng,Update on MSG,New subscribers to our e-newsletter always rec...,2011-02-08T14:48:31Z,2011-02-08,2011,"""Update on MSG"" What about MSG? The scientific..."
2567,2595,2595,lD2RzNJYGxQ,Update on Juice Plus+®,New subscribers to our e-newsletter always rec...,2011-02-08T05:41:12Z,2011-02-08,2011,"""Update on Juice Plus+"" What about Juice Plus+..."


In [31]:
qa_set = pd.read_csv('../data/qa_20_2024-04-11')
qa_set

,Unnamed: 0,question,answer,transcript_ids
0,0,best foods for diverticulosis,"The best foods for diverticulosis are those high in dietary fiber. These include whole grains, beans, vegetables, and fruits. Such diets help in softening and bulking up the stool, reducing the strain during bowel movements which can prevent the formation of diverticula or the aggravation of existing ones.","['5A0ChIZsdHY', 'K64v_V2Z3QE']"
1,1,low carb diets safety,"Low carb diets, like the Atkins Diet, have been associated with serious health risks. For example, a man following the Atkins Diet developed severe cardiovascular issues and erectile dysfunction after starting the diet. His health improved only after switching to a low-fat diet rich in vegetables, grains, and beans.",['Ka_ngjez9dA']
2,2,fiber sources for diverticulosis,"Good sources of fiber for managing diverticulosis include large platefuls of leafy vegetables, whole grains, and legumes. These foods help in maintaining a soft stool consistency, reducing the need for straining during bowel movements and preventing the formation of diverticula.","['5A0ChIZsdHY', 'K64v_V2Z3QE']"
3,3,impact of plant-based diets on cardiovascular disease,Plant-based diets have been shown to reverse heart disease and significantly reduce the risk of cardiovascular issues. This is due to the low levels of cholesterol and saturated fats in plant-based foods compared to diets containing meat and dairy products.,"['c7DZ4OFxONc', 'EvcYy8_bCx0']"
4,4,relation between B12 deficiency and cardiovascular risks,"Vitamin B12 deficiency, common in strict vegetarians who don't supplement, can lead to high homocysteine levels, which may increase the risk of both heart disease and stroke. Supplementing with B12 can help maintain healthy homocysteine levels, reducing these risks.",['Vlr7aXD5hqI']
5,5,role of fiber in preventing diverticulosis,Fiber plays a crucial role in preventing diverticulosis by softening the stool and reducing the pressures needed to move stool through the intestines. This helps prevent the formation of diverticula by reducing the strain and pressure on the intestinal walls.,"['5A0ChIZsdHY', 'K64v_V2Z3QE']"
6,6,effects of high calorie density foods on obesity,"High calorie density foods, such as those high in fats and sugars, contribute to obesity as they provide more calories per bite, which can lead to overeating. Our natural preferences for these calorie-dense foods were suitable for survival in the past, but cause health issues like obesity in modern contexts.",['zMhkwFnoqnc']
7,7,difference between natural and processed foods in calorie sensing,"Humans have evolved to be sensitive to natural variations in food calorie densities, such as between different fruits and vegetables. However, processed foods with very high calorie densities, like cheese and chocolate, dull our natural calorie sensing abilities, leading to overconsumption.",['zMhkwFnoqnc']
8,8,link between soul food and health disparities,"Soul food, rich in added fats, sugars, and salt, has contributed to an excess burden of health issues like high blood pressure and diabetes in the African American community. However, the shift towards healthier, plant-based versions of soul food is helping to address these disparities.",['EvcYy8_bCx0']
9,9,prevention of diverticulosis with nuts,"Contrary to previous beliefs, consuming nuts does not increase the risk of diverticulosis. In fact, a study in the Journal of the American Medical Association found that individuals with diverticulosis who ate nuts had lower rates of inflammation.",['kYMbK4pMYNo']


# Set Up RAG Chain

In [14]:
loader = DataFrameLoader(df_transcripts, 'transcript')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

  0%|          | 0/342 [00:00<?, ?it/s]

  0%|          | 0/342 [00:00<?, ?it/s]

  0%|          | 0/342 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

In [24]:
# Let's get a pre-written prompt from the langchain hub
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [36]:
# Set up the RAG Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain_4 = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chat_model_4
    | StrOutputParser()
)

rag_chain_3 = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chat_model_3
    | StrOutputParser()
)

# Give the RAG Chain a try!

In [37]:
index = 0
question = qa_set.loc[index, "question"]
answer = qa_set.loc[index, "answer"]
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f"GPT3: {rag_chain_3.invoke(question)}")
print(f"GPT4: {rag_chain_4.invoke(question)}")

Question: best foods for diverticulosis
Answer: The best foods for diverticulosis are those high in dietary fiber. These include whole grains, beans, vegetables, and fruits. Such diets help in softening and bulking up the stool, reducing the strain during bowel movements which can prevent the formation of diverticula or the aggravation of existing ones.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT3: The best foods for diverticulosis are high fiber foods like legumes and whole intact greens. These foods help bulk up your stool and prevent excessive pressure on the colon, reducing the risk of diverticulosis. It's important to focus on whole plant foods for a high fiber diet to prevent and manage diverticulitis.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT4: The best foods for managing diverticulosis are high in fiber, particularly legumes and whole intact grains, as they help bulk up the stool and reduce the pressure on the colon. While fruits and vegetables are healthy, they contain a higher water content and less fiber compared to drier foods like whole grains and legumes. It's important to focus on these fiber-rich foods to prevent the progression of diverticulosis and reduce the risk of it becoming inflamed into diverticulitis.


GPT 3.5 says leafy greens instead of leafy grains. I believe GPT4 must have made that correction from the transcript

In [38]:
index = 1
question = qa_set.loc[index, "question"]
answer = qa_set.loc[index, "answer"]
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f"GPT3: {rag_chain_3.invoke(question)}")
print(f"GPT4: {rag_chain_4.invoke(question)}")

Question: low carb diets safety
Answer: Low carb diets, like the Atkins Diet, have been associated with serious health risks. For example, a man following the Atkins Diet developed severe cardiovascular issues and erectile dysfunction after starting the diet. His health improved only after switching to a low-fat diet rich in vegetables, grains, and beans.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT3: Low-carb diets have been associated with higher rates of all-cause mortality, decreased artery function, and other negative side effects. Nutrient deficiencies are a concern with ketogenic diets, as they may not provide enough essential vitamins and minerals. Plant-based diets are considered more nutrient-dense and may offer long-term health benefits compared to low-carb diets.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT4: Low carb diets, such as ketogenic diets, can be associated with safety concerns including nutrient deficiencies, particularly in essential vitamins and minerals. Studies have also shown that low-carb diets may lead to higher rates of all-cause mortality, decreased artery function, and worsening of coronary artery disease, along with other side effects like constipation, headaches, and muscle cramps. Therefore, while effective for weight loss, low carb diets may pose significant health risks and may not be as beneficial for long-term health and longevity compared to more balanced, plant-based diets.


In this example - the answer using the vector search is much better as it is likely using more sources than what I provided in the small test set

In [39]:
index = 2
question = qa_set.loc[index, "question"]
answer = qa_set.loc[index, "answer"]
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f"GPT3: {rag_chain_3.invoke(question)}")
print(f"GPT4: {rag_chain_4.invoke(question)}")

Question: fiber sources for diverticulosis
Answer: Good sources of fiber for managing diverticulosis include large platefuls of leafy vegetables, whole grains, and legumes. These foods help in maintaining a soft stool consistency, reducing the need for straining during bowel movements and preventing the formation of diverticula.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT3: High fiber foods such as legumes and whole intact greens are good sources for diverticulosis prevention. Fruits and vegetables, although healthy, may not provide enough fiber compared to whole grains and legumes. A high fiber diet is recommended to prevent diverticulosis.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT4: For managing diverticulosis, high fiber foods are recommended to help bulk up the stool and reduce straining during bowel movements. Specifically, legumes and whole intact grains are excellent sources of fiber because they are less water-dense compared to fruits and vegetables. However, a study from the University of North Carolina suggested that a high fiber diet might not protect against diverticulosis, although this finding has been debated due to potential flaws in the study.


In this case GPT4 goes a bit off topic into a study about how fiber might not prevent diverticulosis, but the question was just asking about fiber sources. So including that was not helpful. I have to improve the prompt perhaps

In [40]:
index = 3
question = qa_set.loc[index, "question"]
answer = qa_set.loc[index, "answer"]
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f"GPT3: {rag_chain_3.invoke(question)}")
print(f"GPT4: {rag_chain_4.invoke(question)}")

Question: impact of plant-based diets on cardiovascular disease
Answer: Plant-based diets have been shown to reverse heart disease and significantly reduce the risk of cardiovascular issues. This is due to the low levels of cholesterol and saturated fats in plant-based foods compared to diets containing meat and dairy products.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT3: Plant-based diets are associated with a lower risk of cardiovascular disease mortality and overall death rates. They have been linked to lower LDL cholesterol, triglycerides, blood sugars, blood pressures, inflammation, and carotid artery wall thickening. While plant-based diets can reduce the risk of heart disease, stroke risk may still be a concern that needs to be addressed with additional measures like restricting salt intake.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT4: Plant-based diets have been shown to significantly reduce the risk of cardiovascular disease. They can lead to lower LDL cholesterol, triglycerides, blood pressure, and inflammation, and can even arrest or reverse the progression of cardiovascular disease. Increasing the intake of plant foods while reducing animal foods is associated with benefits on cardiovascular health and mortality.


GPT4 and GPT35 have a much better answers here than the one created from the test set. The protective effect of plant based diets is not just due to the low levels of cholesterol and saturated fats

In [41]:
index = 4
question = qa_set.loc[index, "question"]
answer = qa_set.loc[index, "answer"]
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f"GPT3: {rag_chain_3.invoke(question)}")
print(f"GPT4: {rag_chain_4.invoke(question)}")

Question: relation between B12 deficiency and cardiovascular risks
Answer: Vitamin B12 deficiency, common in strict vegetarians who don't supplement, can lead to high homocysteine levels, which may increase the risk of both heart disease and stroke. Supplementing with B12 can help maintain healthy homocysteine levels, reducing these risks.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT3: Vitamin B12 deficiency can lead to increased homocysteine levels, which may contribute to cardiovascular risks. Vegetarians who do not have a reliable source of B12 are at risk for deficiency, potentially impacting their cardiovascular health. Efforts to correct B12 deficiency in vegetarian diets are crucial to prevent associated health risks.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT4: Vitamin B12 deficiency in vegetarians can lead to higher homocysteine levels, which is a known risk factor for cardiovascular disease. Although no significant differences in atherosclerosis were found between vegetarians and non-vegetarians in some studies, the deficiency may negate some cardiovascular benefits of a vegetarian diet, such as improved cholesterol and blood sugar levels. Therefore, it is crucial for vegetarians to ensure adequate intake of vitamin B12 to mitigate these risks.


In [42]:
index = 5
question = qa_set.loc[index, "question"]
answer = qa_set.loc[index, "answer"]
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f"GPT3: {rag_chain_3.invoke(question)}")
print(f"GPT4: {rag_chain_4.invoke(question)}")

Question: role of fiber in preventing diverticulosis
Answer: Fiber plays a crucial role in preventing diverticulosis by softening the stool and reducing the pressures needed to move stool through the intestines. This helps prevent the formation of diverticula by reducing the strain and pressure on the intestinal walls.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT3: A high-fiber diet is associated with a lower risk of diverticulosis, as populations with high-fiber diets have lower rates of the condition. Consuming a vegetarian diet and a high intake of dietary fiber have been shown to lower the risk of hospitalization and death from diverticular disease. Eating whole plant foods, including leafy vegetables, can help prevent diverticulosis.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT4: The role of fiber in preventing diverticulosis is supported by evidence suggesting that high-fiber diets, particularly those rich in whole plant foods, are associated with lower rates of diverticulosis. Studies comparing different populations indicate that those consuming diets high in fiber, such as traditional African diets with 70 to 90 grams of fiber daily, show significantly lower prevalence of diverticulosis compared to Western populations with lower fiber intake. Therefore, a high intake of dietary fiber appears to play a preventive role against the development of diverticulosis.


In this case the GPTs did not say why ie. softening stool and reducing pressure on intestines

In [43]:
index = 6
question = qa_set.loc[index, "question"]
answer = qa_set.loc[index, "answer"]
print(f"Question: {question}")
print(f"Answer: {answer}")
print(f"GPT3: {rag_chain_3.invoke(question)}")
print(f"GPT4: {rag_chain_4.invoke(question)}")

Question: effects of high calorie density foods on obesity
Answer: High calorie density foods, such as those high in fats and sugars, contribute to obesity as they provide more calories per bite, which can lead to overeating. Our natural preferences for these calorie-dense foods were suitable for survival in the past, but cause health issues like obesity in modern contexts.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT3: High calorie density foods, such as highly processed foods like doughnuts, cheese, and meat, can lead to problematic eating behaviors and obesity. Fruits and vegetables, on the other hand, are less related to cravings and loss of control. Foods high in fiber and water are generally lower in energy density and can help with weight control.


  0%|          | 0/1 [00:00<?, ?it/s]

GPT4: High calorie density foods, which are often high in sugar and fat, contribute to obesity because they provide a lot of calories in small amounts of food, leading to excessive calorie intake and weight gain. Foods like doughnuts, cheese, and meat are typically high in calorie density and are associated with cravings and loss of control in eating behaviors. Conversely, low calorie density foods like fruits and vegetables are less likely to contribute to obesity and are not commonly associated with problematic eating behaviors.


This is a pretty low ball question - effects of high calorie density foods on obesity! Pretty obvious I think

# Set up RAG Chain with source retrieval

In [46]:
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | chat_model_4
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke("What is Task Decomposition")

  0%|          | 0/1 [00:00<?, ?it/s]

{'context': [Document(page_content='through autophagy. See, at any given time, most of our cells are producing and assembling more than 10,000 distinct proteins, each of which can become misfolded or damaged at any time and require a cleanup in aisle three. But during times of nutrient excess, our body figures why bother? We can just toss it in the corner and make another. Having evolved in a context of scarcity, our body expects to fall on hard times any day now and can put off spring cleaning until then. But these days, these lean times hardly ever come; so, our cells just end up continually hoarding junk. That’s where autophagy comes from, from the Greek words for “self-eating.” It’s a housekeeping process by which defective cellular components are broken down and scrapped for spare parts. This doubles as both salvage operation and quality control, clearing out some of the damaged debris implicated in the aging process, while renewing our cells in a sort of cellular reset. As one re

In [61]:
index = 5
question = qa_set.loc[index, "question"]
answer = qa_set.loc[index, "answer"]
print(f"Question: {question}")
print(f"Answer: {answer}")
# print(f"GPT3: {rag_chain_3.invoke(question)}")
rag_result = rag_chain_with_source.invoke(question)
rag_result
# print(f"GPT4: ")

Question: role of fiber in preventing diverticulosis
Answer: Fiber plays a crucial role in preventing diverticulosis by softening the stool and reducing the pressures needed to move stool through the intestines. This helps prevent the formation of diverticula by reducing the strain and pressure on the intestinal walls.


  0%|          | 0/1 [00:00<?, ?it/s]

{'context': [Document(page_content='"Does Fiber Really Prevent Diverticulosis?" A study out of the University of North Carolina found no association between dietary fiber intake and diverticulosis in comparing the group that ate the highest amount, 25 grams, three times the amount of the lowest fiber intake group. They concluded that a low fiber diet is not associated with diverticulosis. The university sent out a press release: "Diets high in fiber won\'t protect against diverticulosis." The press picked it up. "Study finds high fiber diet may not protect against diverticulosis." Went all over the Paleo blogs, and even medical journals: an important paper calling into question the fiber theory of the development of diverticulosis. Other editorials, though, caught the critical flaw. To understand this, let\'s turn to another dietary deficiency disease, scurvy. Medical experiments on prisoners in Iowa State Penitentiary showed that clinical signs of scurvy start appearing just 29 days w

In [62]:
rag_result

{'context': [Document(page_content='"Does Fiber Really Prevent Diverticulosis?" A study out of the University of North Carolina found no association between dietary fiber intake and diverticulosis in comparing the group that ate the highest amount, 25 grams, three times the amount of the lowest fiber intake group. They concluded that a low fiber diet is not associated with diverticulosis. The university sent out a press release: "Diets high in fiber won\'t protect against diverticulosis." The press picked it up. "Study finds high fiber diet may not protect against diverticulosis." Went all over the Paleo blogs, and even medical journals: an important paper calling into question the fiber theory of the development of diverticulosis. Other editorials, though, caught the critical flaw. To understand this, let\'s turn to another dietary deficiency disease, scurvy. Medical experiments on prisoners in Iowa State Penitentiary showed that clinical signs of scurvy start appearing just 29 days w

In [63]:
def format_docs(docs) -> str:
    """Convert Documents to a single string.:"""
    formatted = [
        f"Article Title: {doc.metadata['title']}\nArticle Snippet: {doc.page_content}"
        for doc in docs
    ]
    sources = "\n\n" + "\n\n".join(formatted)

    return sources

def format_response(result):
    sources = format_docs(result['context'])
    answer = f"\n\nAnswer: {result['answer']}"

    response = sources + answer
    return response



In [64]:
print(format_response(rag_result))



Article Title: Does Fiber Really Prevent Diverticulosis?
Article Snippet: "Does Fiber Really Prevent Diverticulosis?" A study out of the University of North Carolina found no association between dietary fiber intake and diverticulosis in comparing the group that ate the highest amount, 25 grams, three times the amount of the lowest fiber intake group. They concluded that a low fiber diet is not associated with diverticulosis. The university sent out a press release: "Diets high in fiber won't protect against diverticulosis." The press picked it up. "Study finds high fiber diet may not protect against diverticulosis." Went all over the Paleo blogs, and even medical journals: an important paper calling into question the fiber theory of the development of diverticulosis. Other editorials, though, caught the critical flaw. To understand this, let's turn to another dietary deficiency disease, scurvy. Medical experiments on prisoners in Iowa State Penitentiary showed that clinical signs of

In [57]:
print(format_docs(rag_result['context']))



Article Title: Does Fiber Really Prevent Diverticulosis?
Article Snippet: "Does Fiber Really Prevent Diverticulosis?" A study out of the University of North Carolina found no association between dietary fiber intake and diverticulosis in comparing the group that ate the highest amount, 25 grams, three times the amount of the lowest fiber intake group. They concluded that a low fiber diet is not associated with diverticulosis. The university sent out a press release: "Diets high in fiber won't protect against diverticulosis." The press picked it up. "Study finds high fiber diet may not protect against diverticulosis." Went all over the Paleo blogs, and even medical journals: an important paper calling into question the fiber theory of the development of diverticulosis. Other editorials, though, caught the critical flaw. To understand this, let's turn to another dietary deficiency disease, scurvy. Medical experiments on prisoners in Iowa State Penitentiary showed that clinical signs of